## Variable magnitude

### Does the magnitude of the variable matter?

In Linear Regression models, the scale of variables used to estimate the output matters. Linear models are of the type **y = w x + b**, where the regression coefficient w represents the expected change in y for a one unit change in x (the predictor). Thus, the magnitude of **w** is partly determined by the magnitude of the units being used for **x**. If x is a distance variable, just changing the scale from kilometers to miles will cause a change in the magnitude of the coefficient.

In addition, in situations where we estimate the outcome y by contemplating multiple predictors x1, x2, ...xn, predictors with greater numeric ranges dominate over those with smaller numeric ranges.

Gradient descent converges faster when all the predictors (x1 to xn) are within a similar scale, therefore making feature scaling useful for Neural Networks as well as Logistic Regression.

In Support Vector Machines, feature scaling can decrease the time to find the support vectors.

Finally, methods using Euclidean distances or distances in general are also affected by the magnitude of the features, as Euclidean distance is sensitive to variations in the magnitude or scales of the predictors. Therefore feature scaling is required for methods that utilise distance calculations like k-nearest neighbours (KNN) and k-means clustering.

For more details on the above, follow the links in the Bonus Lecture of this section.

In summary:

#### Maginutd matters because:

- The regression coefficient is directly influenced by the scale of the variable
- Variables with bigger magnitudes /  value range dominate over the ones with smaller magnitudes / value range
- Gradient descent converges faster when features are on similar scales
- Feature scaling helps decrease the time to find support vectors for SVMs
- Euclidean distances are sensitive to feature magnitude.

#### The machine learning models affected by the magnitude of the feature are:

- Linear and Logistic Regression
- Neural Networks
- Support Vector Machines
- KNN
- K-means clustering
- Linear Discriminant Analysis (LDA)
- Principal Component Analysis (PCA)

#### Machine learning models insensitive to feature magnitude are the ones based on Trees:

- Classification and Regression Trees
- Random Forests
- Gradient Boosted Trees


**For more information on whether and why you should scale features prior to use in machine learning models, refer to the lecture "Bonus: Additional reading resources on variable problems", in the previous section of this course.** 



===================================================================================================

## Real Life example: 

### Predicting Survival on the Titanic: understanding society behaviour and beliefs

Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

====================================================================================================

To download the Titanic data, go ahead to this website:
https://www.kaggle.com/c/titanic/data

Click on the link 'train.csv', and then click the 'download' blue button towards the right of the screen, to download the dataset. Save it in a folder of your choice.

**Note that you need to be logged in to Kaggle in order to download the datasets**.

If you save it in the same directory from which you are running this notebook, and you rename the file to 'titanic.csv' then you can load it the same way I will load it below.

====================================================================================================

In this notebook, I will demonstrate the effect of feature magnitude on the performance of different machine learning algorithms.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
% matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

### Load data with numerical variables only

In [2]:
# load the numerical variables of the Titanic Dataset
data = pd.read_csv('titanic.csv', usecols = ['Pclass', 'Age', 'Fare', 'Survived'])
data.head()

,Survived,Pclass,Age,Fare
0,0,3,22.0,7.2500
1,1,1,38.0,71.2833
2,1,3,26.0,7.9250
3,1,1,35.0,53.1000
4,0,3,35.0,8.0500


In [3]:
# let's have a look at the values of those variables to get an idea of the magnitudes

data.describe()

,Survived,Pclass,Age,Fare
count,891.000000,891.000000,714.000000,891.000000
mean,0.383838,2.308642,29.699118,32.204208
std,0.486592,0.836071,14.526497,49.693429
min,0.000000,1.000000,0.420000,0.000000
25%,0.000000,2.000000,20.125000,7.910400
50%,0.000000,3.000000,28.000000,14.454200
75%,1.000000,3.000000,38.000000,31.000000
max,1.000000,3.000000,80.000000,512.329200


In [4]:
# let's now calculate the range

for col in ['Pclass', 'Age', 'Fare']:
    print(col, '_range: ', data[col].max()-data[col].min())

Pclass _range:  2
Age _range:  79.58
Fare _range:  512.3292


The magnitude of the values of the 3 different variables and their ranges of values are quite different. Therefore, feature scaling could benefit the performance of several machine learning algorithms.

In [5]:
# let's separate into training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    data[['Pclass', 'Age', 'Fare']].fillna(0),
    data.Survived,
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((623, 3), (268, 3))

### Feature Scaling

For this demonstration, I will scale the features between 0 and 1. 
To learn more about this scaling visit the scikit-learn website: 
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

Briefly, the transformation is given by:

X_std = (X - X.min() / (X.max - X.min())

And to transform the scaled feature back to its initial format:

X_scaled = X_std * (max - min) + min


**We'll see more on feature scaling in future lectures**

In [6]:
# scaling the features between 0 and 1. 

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
#let's have a look at the scaled training dataset
print('Mean: ', X_train_scaled.mean(axis=0))
print('Standard Deviation: ', X_train_scaled.std(axis=0))
print('Minimum value: ', X_train_scaled.min(axis=0))
print('Maximum value: ', X_train_scaled.max(axis=0))

Mean:  [ 0.64365971  0.30131421  0.06335433]
Standard Deviation:  [ 0.41999093  0.21983527  0.09411705]
Minimum value:  [ 0.  0.  0.]
Maximum value:  [ 1.  1.  1.]


### Logistic Regression

In [8]:
# model build on unscaled variables

logit = LogisticRegression(random_state=44, C=1000) # c big to avoid regularization
logit.fit(X_train, y_train)
print('Train set')
pred = logit.predict_proba(X_train)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = logit.predict_proba(X_test)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Logistic Regression roc-auc: 0.7134823539619531
Test set
Logistic Regression roc-auc: 0.7080952380952381


In [9]:
logit.coef_

array([[-0.92574443, -0.01822382,  0.00233696]])

In [10]:
# model built on scaled variables
logit = LogisticRegression(random_state=44, C=1000) # c big to avoid regularization
logit.fit(X_train_scaled, y_train)
print('Train set')
pred = logit.predict_proba(X_train_scaled)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = logit.predict_proba(X_test_scaled)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Logistic Regression roc-auc: 0.7134931997136722
Test set
Logistic Regression roc-auc: 0.7080952380952381


In [11]:
logit.coef_

array([[-1.85168371, -1.45774407,  1.1951952 ]])

We observe that the performance of logistic regression did not change when using the datasets with the features scaled (compare roc-auc values for train and test set for models with and without feature scaling). 

However, when looking at the coefficients we do see a big difference in the values. This is because the magnitude of the variable was affecting the coefficients. After scaling, all 3 variables have the relative same effect (coefficient) for survival, whereas before scaling, we would be inclined to think that PClass was driving the Survival outcome.

### Support Vector Machines

In [12]:
# model build on data with plenty of categories in Cabin variable

SVM_model = SVC(random_state=44, probability=True)
SVM_model.fit(X_train, y_train)
print('Train set')
pred = SVM_model.predict_proba(X_train)
print('SVM roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = SVM_model.predict_proba(X_test)
print('SVM roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
SVM roc-auc: 0.9016995292943752
Test set
SVM roc-auc: 0.6768154761904762


In [13]:
SVM_model = SVC(random_state=44, probability=True)
SVM_model.fit(X_train_scaled, y_train)
print('Train set')
pred = SVM_model.predict_proba(X_train_scaled)
print('SVM roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = SVM_model.predict_proba(X_test_scaled)
print('SVM roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
SVM roc-auc: 0.7047081408212403
Test set
SVM roc-auc: 0.6988690476190477


Feature scaling improved the performance of the support vector machine. After feature scaling the model is no longer over-fitting to the training set (compare the roc-auc of 0.901 for the model on unscaled features vs the roc-auc of 0.704). In addition, the roc-auc for the testing set increased as well (0.67 vs 0.69).

### Neural Networks

In [14]:
# model built on unscaled features

NN_model = MLPClassifier(random_state=44, solver='sgd')
NN_model.fit(X_train, y_train)
print('Train set')
pred = NN_model.predict_proba(X_train)
print('Neural Network roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = NN_model.predict_proba(X_test)
print('Neural Network roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Neural Network roc-auc: 0.678190277868159
Test set
Neural Network roc-auc: 0.666547619047619


In [15]:
# model built on scaled features

NN_model = MLPClassifier(random_state=44, solver='sgd')
NN_model.fit(X_train_scaled, y_train)
print('Train set')
pred = NN_model.predict_proba(X_train_scaled)
print('Neural Network roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = NN_model.predict_proba(X_test_scaled)
print('Neural Network roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Neural Network roc-auc: 0.7161937918917161
Test set
Neural Network roc-auc: 0.7125


We observe that scaling the features improved the performance of the neural network both for the training and the testing set (compare roc-auc values: training 0.67 vs 0.71; testing: 0.66 vs 0.71). The roc-auc increases in both training and testing sets when the model is trained on a dataset with scaled features.

### K-Nearest Neighbours

In [16]:
#model built on unscaled features

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_train, y_train)
print('Train set')
pred = KNN.predict_proba(X_train)
print('KNN roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = KNN.predict_proba(X_test)
print('KNN roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
KNN roc-auc: 0.8694225721784778
Test set
KNN roc-auc: 0.6253571428571428


In [17]:
# model built on scaled

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_train_scaled, y_train)
print('Train set')
pred = KNN.predict_proba(X_train_scaled)
print('KNN roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = KNN.predict_proba(X_test_scaled)
print('KNN roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
KNN roc-auc: 0.8880555736318084
Test set
KNN roc-auc: 0.7017559523809525


We observe for KNN as well that feature scaling improved the performance of the model. The model built on unscaled features shows a better generalisation, with a higher roc-auc for the testing set (0.70 vs 0.62 for model built on unscaled features).

Both KNN methods are over-fitting to the train set. Thus, we would need to change the parameters of the model or use less features to try and decrease over-fitting, which exceeds the purpose of this demonstration.

### Random Forests

In [18]:
# model built on unscaled features

rf = RandomForestClassifier(n_estimators=700, random_state=39)
rf.fit(X_train, y_train)
print('Train set')
pred = rf.predict_proba(X_train)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = rf.predict_proba(X_test)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Random Forests roc-auc: 0.9914589705212468
Test set
Random Forests roc-auc: 0.7602678571428572


In [19]:
# model built in scaled features
rf = RandomForestClassifier(n_estimators=700, random_state=39)
rf.fit(X_train_scaled, y_train)
print('Train set')
pred = rf.predict_proba(X_train_scaled)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = rf.predict_proba(X_test_scaled)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Random Forests roc-auc: 0.9914589705212469
Test set
Random Forests roc-auc: 0.7602380952380952


As expected, Random Forests shows no change in performance regardless of whether it is trained on a dataset with scaled or unscaled features

In [20]:
ada = AdaBoostClassifier(n_estimators=200, random_state=44)
ada.fit(X_train, y_train)
print('Train set')
pred = ada.predict_proba(X_train)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = ada.predict_proba(X_test)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
AdaBoost roc-auc: 0.847736491616234
Test set
AdaBoost roc-auc: 0.7733630952380953


In [21]:
ada = AdaBoostClassifier(n_estimators=200, random_state=44)
ada.fit(X_train_scaled, y_train)
print('Train set')
pred = ada.predict_proba(X_train_scaled)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = ada.predict_proba(X_test_scaled)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
AdaBoost roc-auc: 0.847736491616234
Test set
AdaBoost roc-auc: 0.7733630952380953


As expected, AdaBoost shows no change in performance regardless of whether it is trained on a dataset with scaled or unscaled features

**That is all for this demonstration. I hope you enjoyed the notebook, and see you in the next one.**